In [1]:

from sklearn.base import TransformerMixin, BaseEstimator

from collections import Counter , defaultdict

import pandas as pd

pd.set_option('display.max_columns', None)


from pandas import Series as s , DataFrame as df
import numpy as np

import seaborn as sns
# import matplotlib.pyplot as plt

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc


%matplotlib inline
rc["figure.figsize"] = 10,6

import datetime
import datetime as dt

from sklearn.model_selection  import StratifiedKFold
from sklearn.decomposition import PCA

from iteration_utilities import duplicates, unique_everseen

import sys
from itertools import groupby
from operator import itemgetter
from timeit import timeit


## RandomOverSampler to handle imbalanced data
from imblearn.over_sampling import RandomOverSampler # over sampling method 2

Using TensorFlow backend.


In [2]:

#Algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.naive_bayes import MultinomialNB

from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

#preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler, normalize



from sklearn.model_selection import train_test_split

import lightgbm as lgb


#Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support

from sklearn.pipeline import Pipeline

# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv("train_Wc8LBpr.csv")

df_test = pd.read_csv("test_VsU9xXK.csv")
submission = pd.read_csv("sample_submission_NoPBkjr.csv")

df10 = df.copy()
df.shape, df_test.shape, submission.shape

((131662, 14), (87395, 13), (87395, 2))

In [4]:
def get_percentage_miss_value(dataset):
    higher_miss_value_column = []
    miss_threshold_value = 50
    
    for i in dataset.columns:
        if dataset[i].isna().sum() > 1: 
            perectange_val = (dataset[i].isna().sum() / len(dataset)) * 100
            print("Column-> " , i, ", total no of missing value : ",dataset[i].isna().sum() , " & :         ", round(perectange_val,2) ," %")
                
            if(perectange_val > miss_threshold_value):
                higher_miss_value_column.append(i)
            
    print("\n\n\n")
    
    if higher_miss_value_column:
        print("Higher Missing values in Columns for Delete : ", higher_miss_value_column)
    else:
        print("There are no Higher Column Missing values in Dataset")

In [5]:
# get_percentage_miss_value(df)

In [6]:
# get_percentage_miss_value(df_test)

In [7]:
# df.info()

In [8]:
def check_cloumn_details_type_categorical(dataset):
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            
            print("*"*100)
            print("\n")

In [134]:
# check_cloumn_details_type_categorical(df)

In [10]:
93900 / len(df)

0.7131898345764154

In [11]:
# check_cloumn_details_type_categorical(df_test)

In [12]:
def check_cloumn_details_type_numberical(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "int"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
            
def check_cloumn_details_type_float(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "float"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            

In [13]:
# check_cloumn_details_type_numberical(df)

In [14]:
# check_cloumn_details_type_float(df)

In [15]:
# Visulazing the distibution of the data for every feature
def visualize_hist(dataset):
    dataset.hist(edgecolor='black', linewidth=1.2, figsize=(20, 20));
    plt.show()

In [16]:
# visualize_hist(df)

In [17]:
def visualize_histogram(dataset):
    # plot histogram
    plt.figure(figsize=(25, 9))  # figure size in ratio 16:9
    features = dataset.columns  # list of columns name
    for i, j in enumerate(features):
        plt.subplot(3, 3, i + 1)  # create subplot for histogram
        plt.title("Histogram of {}".format(j), fontsize=15)  # title of histogram

        bins = len(dataset[j].unique())  # bins for histogram
        plt.hist(dataset[j], bins=bins, rwidth=0.8, edgecolor="y", linewidth=2, )  # plot histogram

    plt.subplots_adjust(hspace=0.5)  # space between horixontal axes (subplots)

In [18]:
# visualize_histogram(df)

In [85]:

    
def corr_metrix(dataset):
    corr = dataset.corr()
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap='RdYlGn')

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
    ax.tick_params(labelsize=20)

def corr_2_more_visualize(dataset):
    corr = dataset.corr()
    
    f, ax = plt.subplots(figsize=(20, 9))
    sns.heatmap(corr.apply(lambda x : np.round(x,2)), 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,annot=True,cmap='RdYlGn', annot_kws={"size": 15})
    ax.tick_params(labelsize=20)
    plt.show()
    
    
def linear_regression_train_test():
    #Linear regression with L2 regularization
    for i in range(-2, 3):
        alpha = 10**i
        rm = Ridge(alpha = alpha)
        ridge_model = rm.fit(X_train, y_train)
        preds_ridge = ridge_model.predict(X_test)
    
        plt.scatter(preds_ridge, y_test, alpha= 0.75, c= 'b')
        plt.xlabel('Predicted price')
        plt.ylabel('Actual price')
        plt.title('Ridge redularization with alpha {}'.format(alpha))
        overlay = 'R square: {} \nRMSE: {}'.format(ridge_model.score(X_test, y_test), mean_squared_error(y_test, preds_ridge))
        plt.annotate(s = overlay, xy = (12.1, 10.6), size = 'x-large')
        plt.show()
        
def actual_predict_visualization(actual_values, predict_values):
    plt.scatter(actual_values, predict_values, alpha= 0.75, color = 'b')

    plt.xlabel('Predicted price')
    plt.ylabel('Actual price')
    plt.title('Regression Model')
    plt.show()
    
def visualize_original_predict_test(X_val, Y_val, default = "Test"):
    print("Dataset type  : \n" ,default)
    plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
    plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

    plt.xticks(())
    plt.yticks(())
    
    plt.xlabel("X axis", color = "red")
    plt.ylabel("Y axis", color = "green")

    plt.show()
    
    
def visualize_categorical_values(dataset):
    no_of_columns = 4
    no_of_rows = 4
    
    columns_object_type = [i for i in dataset.columns  if dataset[i].dtype == "object"]
    total_rows = (len(columns_object_type) // no_of_rows ) + 1
    
    f, axes = plt.subplots(total_rows, no_of_columns, figsize=(18,24))

    for ind, val in enumerate(columns_object_type):
        sns.countplot(df[val] , ax = axes[ind // no_of_rows , ind %no_of_columns ])
    plt.show()   
    
# sns.pairplot(data = dataset[['arpu_6','arpu_7','arpu_8','churn_flag']],hue = 'churn_flag')

def visualize_pairplot_target(dataset, numberical_col_list , target_val):
    plt.figure(figsize=(18,34))
    sns.pairplot(data = dataset[numberical_col_list],hue = target_val)
    
    #     sns.pairplot(data = dataset[['arpu_6','arpu_7','arpu_8','churn_flag']],hue = 'churn_flag') 
    #check for classification
    plt.show()
    
    
def visualize_numberical_values(dataset):
    plt.figure(figsize=(18,34))
    sns.pairplot(df)
    plt.show()
# visualize_numberical_values(df)
    
def check_skewness_numerical(dataset, target):
    #analysing the distribution of sale price
    print('skew is', dataset.SalePrice.skew())   
    plt.hist(dataset[target], color= 'b')

    plt.title('Distribution of ' + target, fontsize = 24)
    plt.ylabel('observation', fontsize = 20)
    plt.xlabel(target, fontsize = 20)

    plt.show()
    
def skewness_after_log_transform(dataset , target):
    #log transforming sale price to transform it into gaussian distribution
    target1 = np.log(dataset.target)
    print('skew is', target1.skew())
    plt.hist(target, color= 'b')

    plt.title('Distribution of ' + target, fontsize = 24)
    plt.ylabel('observation', fontsize = 20)
    plt.xlabel(target , fontsize = 20)

    plt.show()
    
# Since pairplots for all numeric values together are not clear we can make groups ,do plot with price & analyse

def pair_plot(list_4_numberical_values):
    sns.pairplot(df2, x_vars= list_4_numberical_values, y_vars='SalePrice',size=4, kind='scatter')
    plt.show()

In [86]:
df.shape

(131662, 14)

In [21]:
# corr_2_more_visualize(df)

In [22]:
# visualize_categorical_values(df)

In [23]:


# visualize_numberical_values(df)

In [24]:
#Label encoding
def convert_to_numerical_label_encoding(dataset):
    enc = LabelEncoder()
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            dataset[i] = enc.fit_transform(dataset[i])
            
    return dataset

In [25]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    


In [45]:
def majority_imbalanced_dataset(dataset, target_col):
    feature_columns = dataset.columns.tolist()
    feature_columns = [c for c in feature_columns if c not in [target_col]]

    X2_new = dataset[feature_columns]
    Y2_new = dataset[target_col]

    os =  RandomOverSampler(random_state=35)
    X_feature_variables , y_output = os.fit_sample(X2_new, Y2_new)
    
    X_feature_variables[target_col] = y_output
    
    X_feature_variables = X_feature_variables.sample(frac = 1).reset_index(drop = True)
    
    return X_feature_variables

In [46]:
def standardscaler_preprocessing(dataset_train, dataset_test, num_col):
    scaler = StandardScaler()
   
    dataset_train[num_col] = scaler.fit_transform(dataset_train[num_col])

    dataset_test[num_col] = scaler.transform(dataset_test[num_col])
    
    return dataset_train, dataset_test

# <font color ='green' > EDA

In [242]:
# drop_col = ["Trip_ID" , "Var1", "Var2", "Var3"]

drop_col = ["Trip_ID", "Var1"  ]

In [243]:
df1 = df.drop(columns = drop_col)
df.shape , df1.shape

((131662, 14), (131662, 12))

In [244]:
df1_test = df_test.drop(columns = drop_col)
df_test.shape , df1_test.shape

((87395, 13), (87395, 11))

In [245]:
check_cloumn_details_type_categorical(df1)

Columns name :   Type_of_Cab
{'B': 31136, nan: 20210, 'C': 28122, 'E': 11634, 'D': 18991, 'A': 21569}
****************************************************************************************************


Columns name :   Confidence_Life_Style_Index
{'A': 35147, 'B': 40355, nan: 20193, 'C': 35967}
****************************************************************************************************


Columns name :   Destination_Type
{'A': 77597, 'E': 2717, 'B': 29555, 'C': 7484, 'G': 1489, 'D': 6588, 'F': 1950, 'K': 680, 'L': 643, 'H': 1260, 'I': 813, 'J': 695, 'M': 93, 'N': 98}
****************************************************************************************************


Columns name :   Gender
{'Female': 37762, 'Male': 93900}
****************************************************************************************************




In [246]:
# df1["Type_of_Cab"] = df1["Type_of_Cab"].fillna("Unkown")
# df1["Confidence_Life_Style_Index"] = df1["Confidence_Life_Style_Index"].fillna("Unkown")

In [247]:
df2 = DataFrameImputer().fit_transform(df1.copy())


df2_test = DataFrameImputer().fit_transform(df1_test.copy())

In [248]:
df2.isna().sum().sum(), df2_test.isna().sum().sum()

(0, 0)

In [249]:
df3 = convert_to_numerical_label_encoding(df2.copy())
df2.shape, df3.shape

((131662, 12), (131662, 12))

In [250]:
df3_test = convert_to_numerical_label_encoding(df2_test.copy())
df2_test.shape, df3_test.shape

((87395, 11), (87395, 11))

In [251]:
Counter(df3.Surge_Pricing_Type)

Counter({2: 56728, 3: 47720, 1: 27214})

In [252]:

# df3 = majority_imbalanced_dataset(df3.copy(), "Surge_Pricing_Type")
# Counter(df4.Surge_Pricing_Type)

In [253]:
df3.shape

(131662, 12)

In [254]:
# scaling_col = ["Trip_Distance" , "Customer_Since_Months" , "Life_Style_Index", "Customer_Rating", "Var2", "Var3"]

scaling_col = ["Trip_Distance" , "Customer_Since_Months" , "Life_Style_Index", "Customer_Rating"]



# df4, df4_test = standardscaler_preprocessing(df3.copy(), df3_test.copy(), scaling_col)

In [255]:
df3.head()

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var2,Var3,Gender,Surge_Pricing_Type
0,6.77,1,1.0,2.427690,0,0,3.90500,0,46,60,0,2
1,29.47,1,10.0,2.782450,1,0,3.45000,0,56,78,1,2
2,41.58,1,10.0,2.802064,1,4,3.50125,2,56,77,1,2
3,61.56,2,10.0,2.802064,1,0,3.45375,0,52,74,1,3
4,54.95,2,10.0,3.034530,1,0,3.40250,4,49,102,1,2


In [256]:
df3.describe()

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var2,Var3,Gender,Surge_Pricing_Type
count,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000
mean,44.200909,1.603340,6.016661,2.802064,1.006228,1.000144,2.849458,0.782838,51.202800,75.099019,0.713190,2.155747
std,25.522882,1.177128,3.544410,0.207760,0.734909,1.906857,0.980675,1.037559,4.986142,11.578278,0.452274,0.738164
min,0.310000,0.000000,0.000000,1.596380,0.000000,0.000000,0.001250,0.000000,40.000000,52.000000,0.000000,1.000000
25%,24.580000,1.000000,3.000000,2.687952,0.000000,0.000000,2.152500,0.000000,48.000000,67.000000,0.000000,2.000000
50%,38.200000,1.000000,6.016661,2.802064,1.000000,0.000000,2.895000,0.000000,50.000000,74.000000,1.000000,2.000000
75%,60.730000,2.000000,10.000000,2.912815,2.000000,1.000000,3.582500,1.000000,54.000000,82.000000,1.000000,3.000000
max,109.230000,4.000000,10.000000,4.875110,2.000000,13.000000,5.000000,8.000000,124.000000,206.000000,1.000000,3.000000


In [257]:
X = df3.iloc[:,:-1]

Y = df3.iloc[:,-1]

In [313]:
from xgboost import XGBClassifier

xg_param_test = {
    
    'gamma': [2, 5,7],
    'max_depth': [3, 4,5],
    'learning_rate': [0.05,0.07],
    'n_estimators': [100,200],
#     objective = ['binary:logistic',  'multi:softmax'],
#     num_classes = [2, 3] # output : binary(2) or multi classification
  
}

XGB_hyper_params = GridSearchCV(estimator = 
XGBClassifier(learning_rate =0.1,
#               objective= 'binary:logistic',
              objective = 'multi:softmax',
                                num_classes =  3,
              nthread=4,
              seed=27), 
                                
              param_grid = xg_param_test,
              scoring= 'accuracy',
              n_jobs=4,
              iid=False,
              verbose=10)



In [314]:
XGB_hyper_params.fit(X,Y)
print(XGB_hyper_params.best_params_)
xgbclf=XGBClassifier(gamma= 7,learning_rate= 0.05, max_depth= 5,n_estimators= 200)
xgbclf.fit(X,Y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 14.5min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 26.6min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 37.9min


KeyboardInterrupt: 

In [ ]:
y_pred_final = xgbclf.predict(df3_test)

In [258]:
from lightgbm import LGBMClassifier

lg_param_test = {
    
    'gamma': [0.5, 0.25],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.07, 0.05],
    'n_estimators': [200, 100]
  
}


LGB_hyper_params = GridSearchCV(estimator = 
LGBMClassifier(learning_rate =0.1,
              nthread=4,
              seed=27), 
              param_grid = lg_param_test,
              scoring= 'accuracy',
              n_jobs=4,
              iid=False,
              verbose=10)


In [259]:
LGB_hyper_params.fit(X,Y)


print(LGB_hyper_params.best_params_)


lgbclf= LGBMClassifier(gamma= 0.5, learning_rate= 0.05,max_depth= 3, n_estimators= 100)


lgbclf.fit(X,Y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   37.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   42.8s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  4.4min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:  5.8min finished


{'gamma': 0.5, 'learning_rate': 0.07, 'max_depth': 5, 'n_estimators': 200}


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               gamma=0.5, importance_type='split', learning_rate=0.05,
               max_depth=3, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [ ]:
lgbclf= LGBMClassifier(gamma= 0.5, learning_rate= 0.07,max_depth= 5, n_estimators= 200, objective='multi:softmax', num_classes=3)


# lgbclf= LGBMClassifier(gamma= 0.5, learning_rate= 0.07,max_depth= 5, n_estimators= 200)
# Default  objective='multi:softmax', num_classes=3




In [ ]:
lgbclf.fit(X,Y)

In [298]:
# y_pred_final = lgbclf.predict(df3_test)

0.23


In [304]:
from catboost import CatBoostClassifier

cat_param_test = {
    'depth':[2, 3],
    'learning_rate': [0.07, 0.05],
    'n_estimators': [200, 100],
    'loss_function': ['Logloss', 'CrossEntropy', 'MultiClass'],
    'l2_leaf_reg':np.logspace(-25, -20, 3)
  
}

CAT_hyper_params = GridSearchCV(estimator = 
CatBoostClassifier(learning_rate =0.1), 
              param_grid = cat_param_test,
              scoring= 'accuracy',
              n_jobs=4,
                                
              iid=False,
              verbose=10)


In [305]:
%%time

CAT_hyper_params.fit(X,Y)

print(CAT_hyper_params.best_params_)


# catclf= CatBoostClassifier(depth= 3,l2_leaf_reg= 1e-25,learning_rate= 0.05,loss_function= 'MultiClass',n_estimators= 200)
# catclf.fit(X,Y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   16.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   17.4s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   17.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   45.2s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:  5.3min
[Parallel(

0:	learn: 1.0598937	total: 151ms	remaining: 30.1s
1:	learn: 1.0274036	total: 226ms	remaining: 22.4s
2:	learn: 0.9997184	total: 279ms	remaining: 18.3s
3:	learn: 0.9759962	total: 331ms	remaining: 16.2s
4:	learn: 0.9557617	total: 387ms	remaining: 15.1s
5:	learn: 0.9379590	total: 438ms	remaining: 14.1s
6:	learn: 0.9225479	total: 477ms	remaining: 13.2s
7:	learn: 0.9090018	total: 518ms	remaining: 12.4s
8:	learn: 0.8972015	total: 557ms	remaining: 11.8s
9:	learn: 0.8864577	total: 604ms	remaining: 11.5s
10:	learn: 0.8774152	total: 655ms	remaining: 11.2s
11:	learn: 0.8689736	total: 699ms	remaining: 10.9s
12:	learn: 0.8615217	total: 738ms	remaining: 10.6s
13:	learn: 0.8553893	total: 777ms	remaining: 10.3s
14:	learn: 0.8491161	total: 826ms	remaining: 10.2s
15:	learn: 0.8437042	total: 876ms	remaining: 10.1s
16:	learn: 0.8389100	total: 916ms	remaining: 9.86s
17:	learn: 0.8346664	total: 954ms	remaining: 9.65s
18:	learn: 0.8303049	total: 994ms	remaining: 9.47s
19:	learn: 0.8267699	total: 1.04s	remaini

163:	learn: 0.7455445	total: 9.17s	remaining: 2.01s
164:	learn: 0.7454683	total: 9.26s	remaining: 1.96s
165:	learn: 0.7453850	total: 9.31s	remaining: 1.91s
166:	learn: 0.7452979	total: 9.35s	remaining: 1.85s
167:	learn: 0.7451806	total: 9.4s	remaining: 1.79s
168:	learn: 0.7450085	total: 9.45s	remaining: 1.73s
169:	learn: 0.7448863	total: 9.49s	remaining: 1.67s
170:	learn: 0.7448014	total: 9.53s	remaining: 1.62s
171:	learn: 0.7447110	total: 9.57s	remaining: 1.56s
172:	learn: 0.7446168	total: 9.62s	remaining: 1.5s
173:	learn: 0.7445200	total: 9.67s	remaining: 1.44s
174:	learn: 0.7444351	total: 9.71s	remaining: 1.39s
175:	learn: 0.7443600	total: 9.75s	remaining: 1.33s
176:	learn: 0.7442480	total: 9.78s	remaining: 1.27s
177:	learn: 0.7441861	total: 9.83s	remaining: 1.21s
178:	learn: 0.7440655	total: 9.88s	remaining: 1.16s
179:	learn: 0.7439422	total: 9.92s	remaining: 1.1s
180:	learn: 0.7438605	total: 9.96s	remaining: 1.04s
181:	learn: 0.7437873	total: 10s	remaining: 989ms
182:	learn: 0.743

In [ ]:
{'depth': 3, 'l2_leaf_reg': 1e-25, 'learning_rate': 0.07, 'loss_function': 'MultiClass', 'n_estimators': 200}
CPU times: user 31.7 s, sys: 1.59 s, total: 33.3 s

In [306]:
catclf= CatBoostClassifier(depth= 3,l2_leaf_reg= 1e-25,learning_rate= 0.07,loss_function= 'MultiClass',n_estimators= 200)



catclf.fit(X,Y)

0:	learn: 1.0598937	total: 40.3ms	remaining: 8.03s
1:	learn: 1.0274036	total: 78.6ms	remaining: 7.79s
2:	learn: 0.9997184	total: 117ms	remaining: 7.68s
3:	learn: 0.9759962	total: 157ms	remaining: 7.67s
4:	learn: 0.9557617	total: 195ms	remaining: 7.62s
5:	learn: 0.9379590	total: 235ms	remaining: 7.58s
6:	learn: 0.9225479	total: 296ms	remaining: 8.17s
7:	learn: 0.9090018	total: 344ms	remaining: 8.25s
8:	learn: 0.8972015	total: 384ms	remaining: 8.14s
9:	learn: 0.8864577	total: 424ms	remaining: 8.05s
10:	learn: 0.8774152	total: 462ms	remaining: 7.94s
11:	learn: 0.8689736	total: 507ms	remaining: 7.95s
12:	learn: 0.8615217	total: 562ms	remaining: 8.09s
13:	learn: 0.8553893	total: 602ms	remaining: 7.99s
14:	learn: 0.8491161	total: 642ms	remaining: 7.91s
15:	learn: 0.8437042	total: 681ms	remaining: 7.83s
16:	learn: 0.8389100	total: 732ms	remaining: 7.88s
17:	learn: 0.8346664	total: 783ms	remaining: 7.92s
18:	learn: 0.8303049	total: 825ms	remaining: 7.86s
19:	learn: 0.8267699	total: 864ms	remai

163:	learn: 0.7455445	total: 7.8s	remaining: 1.71s
164:	learn: 0.7454683	total: 7.85s	remaining: 1.67s
165:	learn: 0.7453850	total: 7.92s	remaining: 1.62s
166:	learn: 0.7452979	total: 7.99s	remaining: 1.58s
167:	learn: 0.7451806	total: 8.04s	remaining: 1.53s
168:	learn: 0.7450085	total: 8.09s	remaining: 1.48s
169:	learn: 0.7448863	total: 8.13s	remaining: 1.44s
170:	learn: 0.7448014	total: 8.18s	remaining: 1.39s
171:	learn: 0.7447110	total: 8.22s	remaining: 1.34s
172:	learn: 0.7446168	total: 8.26s	remaining: 1.29s
173:	learn: 0.7445200	total: 8.31s	remaining: 1.24s
174:	learn: 0.7444351	total: 8.36s	remaining: 1.19s
175:	learn: 0.7443600	total: 8.41s	remaining: 1.15s
176:	learn: 0.7442480	total: 8.45s	remaining: 1.1s
177:	learn: 0.7441861	total: 8.5s	remaining: 1.05s
178:	learn: 0.7440655	total: 8.54s	remaining: 1s
179:	learn: 0.7439422	total: 8.58s	remaining: 954ms
180:	learn: 0.7438605	total: 8.62s	remaining: 905ms
181:	learn: 0.7437873	total: 8.66s	remaining: 857ms
182:	learn: 0.7437

In [307]:
y_pred_final = catclf.predict(df3_test)

In [266]:
y_pred_final = lgbclf.predict(df3_test)

In [238]:
# model_final = XGBClassifier(
#  learning_rate =0.1,
#  n_estimators=1000,
#  max_depth=5,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'multi:softmax',
#  nthread=4,
#  scale_pos_weight=1,
#  num_classes=3,
#  seed=27)

model_final = XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)


model_final.fit(X, Y)

y_pred_final = model_final.predict(df3_test)

In [239]:
"hi"

'hi'

In [308]:
submission_1 = submission.copy()

print(df_test.shape , submission.shape , len(y_pred_final))

submission_1["Surge_Pricing_Type"] = y_pred_final

print(submission_1["Surge_Pricing_Type"].values)


submission_1.to_csv('XGBoost_final_17.csv', index=False)

df_submission_1 = pd.read_csv('XGBoost_final_17.csv')

print(df_submission_1.shape)

df_submission_1.head(3)

(87395, 13) (87395, 2) 87395
[1 2 2 ... 1 2 2]
(87395, 2)


,Trip_ID,Surge_Pricing_Type
0,T0005689459,1
1,T0005689462,2
2,T0005689463,2


In [65]:
print( df["Trip_Distance"].max() , df["Trip_Distance"].min() , df["Trip_Distance"].mean())

print( df["Customer_Since_Months"].max() , df["Customer_Since_Months"].min() , df["Customer_Since_Months"].mean())

print( df["Life_Style_Index"].max() , df["Life_Style_Index"].min() , df["Life_Style_Index"].mean())

print( df["Customer_Rating"].max() , df["Customer_Rating"].min() , df["Customer_Rating"].mean())


109.23 0.31 44.20090853853045
10.0 0.0 6.016661099712109
4.87511 1.59638 2.8020639999461827
5.0 0.00125 2.849457958636489


In [161]:
def get_accuracy(y_train_val , y_pred_val , dataset_type = "Default"):
    
    print(" Dataset type is : ", dataset_type)
    
    print("\n Accuracy Score     : ",round(accuracy_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n precision_accuracy : ",round(precision_score(y_train_val, y_pred_val), 4) * 100)
 
    print("\n recall_accuracy    : ",round(recall_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n roc_auc_accuracy   : ",round(roc_auc_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n f1_score_accuracy  : ",round(f1_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n explained_variance  : ",round(explained_variance_score(y_train_val, y_pred_val), 4) * 100)
    
    tn, fp, fn, tp = confusion_matrix(y_train_val, y_pred_val).ravel()
    
    print("\n Confusion Matrix TN : ", tn, " FP : ", fp, " FN : ", fn, " TP : ", tp)

In [162]:

rows=[]

def addRandomStateForAlgorithm(x,y,names,algorithms,columns_name,random_state_list):    
    for j in range(len(algorithms)):
        model = algorithms[j]
        for i in random_state_list:
            
            x_train, x_test , y_train , y_test = train_test_split(x ,y , test_size = 0.30 , random_state = i)
            
            model.fit(x_train,y_train)
            
            y_pred_train = model.predict(x_train)
            y_pred = model.predict(x_test)
            
            train_acc = accuracy_score(y_train, y_pred_train)
            train_acc = round(train_acc, 4) * 100
            
            test_acc = accuracy_score(y_test, y_pred)
            test_acc = round(test_acc, 4) * 100
            
#             roc_auc_score_acc = roc_auc_score(y_test, y_pred)
#             roc_auc_score_acc = round(roc_auc_score_acc, 4) * 100
            

#             row = [names[j],   i,   train_acc, test_acc, roc_auc_score_acc]
            row = [names[j],   i,   train_acc, test_acc]
    
            rows.append(row)
            
    models_df = pd.DataFrame(rows) 
    
    models_df.columns = columns_name
    print(models_df)
    

In [163]:
   
# names_regression = [ "LightGBM", "RF", "XGBoost" , "SVM"]
algorithms = [ LGBMClassifier(  ), RandomForestClassifier(), XGBClassifier(), SVC()]

# columns_name = ["Model",    "Random_state",   'Train_acc',     "Test_acc" , "roc_auc_score"]

names_regression = [ "XGBClassifier"]
algorithms = [ XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)]

columns_name = ["Model",    "Random_state",   'Train_acc',     "Test_acc" ]


random_state_list_up_to_10 = [1]


addRandomStateForAlgorithm(X,Y,names_regression,algorithms,columns_name,random_state_list_up_to_10)

           Model  Random_state  Train_acc  Test_acc
0  XGBClassifier             1      70.68     70.01


(87395, 13) (87395, 2) 87395
[1 2 2 ... 1 2 2]
(87395, 2)


,Trip_ID,Surge_Pricing_Type
0,T0005689459,1
1,T0005689462,2
2,T0005689463,2


In [100]:
#train : train is a dataset

In [101]:
#Dataset : train & test


In [103]:
df.columns

Index(['Trip_ID', 'Trip_Distance', 'Type_of_Cab', 'Customer_Since_Months',
       'Life_Style_Index', 'Confidence_Life_Style_Index', 'Destination_Type',
       'Customer_Rating', 'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3',
       'Gender', 'Surge_Pricing_Type'],
      dtype='object')

In [104]:
Counter(df.Surge_Pricing_Type)

Counter({2: 56728, 3: 47720, 1: 27214})

In [109]:
df3[0:87395].shape  , df3[87395 : ].shape

((87395, 12), (44267, 12))

In [141]:
def majority_imbalanced_dataset(dataset, target_col):
    feature_columns = dataset.columns.tolist()
    feature_columns = [c for c in feature_columns if c not in [target_col]]

    X2 = dataset[feature_columns]
    Y2 = dataset[target_col]

    os =  RandomOverSampler(random_state=35)
    X_feature_variables , y_output = os.fit_sample(X2, Y2)
    
    return X_feature_variables , y_output

In [142]:
X_feature_variables1 , y_output1 = majority_imbalanced_dataset(df3, "Surge_Pricing_Type")

In [143]:
df10 = X_feature_variables1.copy()

In [145]:
df10["Surge_Pricing_Type"] = y_output1

In [146]:
df10.head()

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var2,Var3,Gender,Surge_Pricing_Type
0,-1.466568,1,-1.415378,-1.801959e+00,0,0,1.076346,0,-1.043456,-1.304086,0,2
1,-0.577167,1,1.123841,-9.440728e-02,1,0,0.612378,0,0.962110,0.250555,1,2
2,-0.102689,1,1.123841,3.633778e-14,1,4,0.664638,2,0.962110,0.164186,1,2
3,0.680141,2,1.123841,3.633778e-14,1,0,0.616202,0,0.159884,-0.094921,1,3
4,0.421157,2,1.123841,1.118919e+00,1,0,0.563942,4,-0.441786,2.323410,1,2


In [148]:
df10.shape , df3.shape

((170184, 12), (131662, 12))

In [162]:
new_test = df10[0:87395]

new_train = df10[87395 : ] 

In [163]:
x1_train = new_train.iloc[:, :-1]
y1_train = new_train.iloc[:, -1]

In [164]:
x1_test = new_test.iloc[:, :-1]
y1_test = new_test.iloc[:, -1]

In [152]:
model_final = XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)


model_final.fit(x1_train, y1_train)


y_pred_final_new_train = model_final.predict(x1_train)

# y_pred_final_new_test = model_final.predict(x1_test)

In [166]:
len(y1_test)

87395

In [167]:
y_pred_final_new_test = model_final.predict(df3_test)

In [ ]:
y_

In [154]:
get_accuracy(y1_train , y_pred_final_new_train ,  "Train")

 Dataset type is :  Train

 Accuracy Score     :  69.41000000000001
[[32209  3577  2880]
 [ 7132  7910  3973]
 [ 3632  4130 17346]]


In [156]:
get_accuracy(y1_test , y_pred_final_new_test ,  "Test")

 Dataset type is :  Test

 Accuracy Score     :  32.190000000000005
[[ 6722  4775  6565]
 [14185  9913 13615]
 [11743  8379 11498]]


In [137]:
get_accuracy(y1_test , y_pred_final_new_test ,  "Their Test")

 Dataset type is :  Their Test

 Accuracy Score     :  37.980000000000004
[[ 2570 10011  5481]
 [ 5323 21140 11250]
 [ 4447 17693  9480]]


In [157]:
def get_accuracy(y_train_val , y_pred_val , dataset_type = "Default"):
    
    print(" Dataset type is : ", dataset_type)
    
    print("\n Accuracy Score     : ",round(accuracy_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n precision_accuracy : ",round(precision_score(y_train_val, y_pred_val), 4) * 100)
 
#     print("\n recall_accuracy    : ",round(recall_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n roc_auc_accuracy   : ",round(roc_auc_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n f1_score_accuracy  : ",round(f1_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n explained_variance  : ",round(explained_variance_score(y_train_val, y_pred_val), 4) * 100)
    
#     tn, fp, fn, tp = confusion_matrix(y_train_val, y_pred_val).ravel()
    
#     print("\n Confusion Matrix TN : ", tn, " FP : ", fp, " FN : ", fn, " TP : ", tp)
    print(confusion_matrix(y_train_val, y_pred_val))

In [158]:
submission_new["Surge_Pricing_Type"] = y_pred_final_new_test


submission_new.to_csv('submission_new_imbalance.csv', index=False)


In [159]:
submission_new.shape , submission.shape

((87395, 2), (87395, 2))